In [ ]:
import ticktack
from jax.numpy import array, pi, exp, sin, mean, median, var, arange, sum
from numpy import random
from pandas import DataFrame
from plotnine import ggplot, facet_wrap, labs, aes, theme_bw, geom_tile, scale_color_cmap
from os import getcwd, walk
import emcee

The basic question is what is the probability of detecting consecutive events based on the distribution of the data. The first step then will be to determine the distribution of the data. This will be done be resampling the points after the event has been removed.

In [ ]:
models = { # This dictionary contains the units for the fluxes and production function
    "Guttler14": {  # Units of the Guttler 2014 paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production rate 
        "flow_rate_units": "Gt/yr"                  # Units of the fluxes
    },
    "Brehm21": {    # Units used by the Brehm, et. al. paper
        "production_rate_units": "kg/yr",    # Units of the production rate
        "flow_rate_units": "Gt/yr"           # Units of the fluxes
    },
    "Buntgen18": {  # The units used by the Buntgen 2018 paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production function
        "flow_rate_units": "Gt/yr"                  # Units of the fluxes 
    },
    "Miyake17": {   # The units used by the Miyake 2017 et. al. paper
        "production_rate_units": "atoms/cm^2/s",    # Units of the production function 
        "flow_rate_units": "1/yr"                   # Units of the fluxes.
    }
}

In [ ]:
data_sets = []  # List for storing the data set locations
data_sets_directory = f"{getcwd()}/datasets" # Home directory of the data 
for (root, dirs, files) in walk(data_sets_directory):    # Looping over directories 
    for file in files:  # Looping through the files 
        file_path = root + "/" + file   # Setting up the path 
        data_sets.append(file_path)  # Extending the stored directoriess

The function below will also have a `shape` parameter eventually. First however I want to get this running. Damn this really is well suited to a class structure since then I can set `self.set_annual_samples()` need to check if this has actually been implemented. The answer is __No__. I need to add the growth seasons to the `model_units` (which I might just rename `models`). This will lead to ?two? extra field `hemisphere_model` (bool) and `growth_seasons`. Alternatively this could result in a further nested dictionary like `hemispheres = {"NH_growth": array([]), "SH_growth": array([])}`

In [ ]:
def get_model(model: str, datum: str):
    cbm = ticktack.load_presaved_model( # Generating the CarbobBoxModel using ticktack
        model,  # Name of the model as looped from the models dictionary 
        production_rate_units=models[model]["production_rate_units"], 
        flow_rate_units=models[model]["flow_rate_units"]
    )

    bayesian_model = ticktack.fitting.SingleFitter(cbm)   # Fitting a model 
    bayesian_model.prepare_function(model="simple_sinusoid")# Generating the simple sin model
    bayesian_model.load_data(datum)   # Loading the data into the model  

    return bayesian_model

In [ ]:
def get_production_function(model: ticktack.fitting.SingleFitter):
    """
    Parameters:
        model: `str` - The `CarbonBoxModel` that is to be used
        data: `str` - The dataset that the production function is to be fitted to (.csv)
    Returns:
        production: `function` - The ideal production function 
    """
    # Return to nedler-mead 
    # Simulate 50 year "template/burn in"
    # Fit the miyake data with mcmc including sinusoid. 
    # Not too comfortable with atmospheric dynamics.
        # Stratospheric pull down ? radio carbon
        # Pulled down close to the surface at mid-year
    # ANU carbon models
        # Cameron O'neil
    # Sharp-rise (Late rise) 774 AD datasets

    # The template that I want to pick removes the sinuspoid but uses the event parameters that were fitted with the sinusoid. Using some measure like MLE or mean. That way we are isolating the event. Save this to a file and evaluate on a 50 year grid with the event happening in the middle. 

    # So split the file into get_template.ipynb -> Run once and save the output onto a 50 year grid. 
    initial_parameters = array([model.time_data[len(model.time_data) // 2], 1./12, pi/2., 81./12])
    # Guess the initial parameters by hand.

    # emcee is an implementation of affine invariant ensamble mcmc. Metropolis hastings is the vanilla that has one walker and each evaluation is a step/not step of the walker. You can also do parallel tempering (MH). Ensamble mcmc is something different, you take a hundred points and propose a dist based on the stretch move. The stretch move draws lines between the points and takes a step along the line (from some dist). This results in the thing being affine, which is a generalisation of linear. This is provably optimal for doing mcmc on a multivariate gaussian.

    # Need to initialise the parameters as a multi-dimensional gaussian noise, to stetch the lines. This is what the condition number means. The condition number is the ratio of the smallest and largest eigenvalue.

    # DFM's website demonstrates how to do this 
    sampler = emcee.EnsembleSampler(4, 4, model.log_likelihood)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(initial_parameters, 200, progress=True);

    print("Running production...")
    sampler.reset()
    sampler.run_mcmc(p0, model.simple_sinusoid, progress=True);
    samples = sampler.flatchain

    parameters = {  # A dictionary of the model parameters for the `simple_sinusoid`
        "Start Date (yr)": None,    # The year that the event began 
        "Duration (yr)": None,      # Number of years that the event occured over 
        "Phase (yr)": None,         # The phase shift of the sinusoidal production function 
        "Area": None               #? What are the units?
    } 

    for i, parameter in enumerate(parameters):  # Looping through the parameters 
        parameters[parameter] = {   # Nested dictionary to store statistical information
            "mean": mean(samples[:, i]),    # Storing the mean of the samples produced by mcmc
            "median": median(samples[:, i]),# Storing the median in addition to the mean 
            "variance": var(samples[:, i])  # Storing the variance of the parameter
        }
    parameters["Steady Production"] = model.steady_state_production   
    
    return parameters

In [ ]:
def production(t: float, params: dict):
    """
    The best fit production function as estimated using `mcmc`
    """
    middle = params["Start Date (yr)"]["mean"] + \
        params["Duration (yr)"]["mean"] / 2.0    # Calculating the center of the event
    height = params["Area"]["mean"] / params["Duration (yr)"]["mean"] # The magnitude of the event 

    gauss = height * exp(- ((t - middle) / (1. / 1.93516 * \
        params["Duration (yr)"]["mean"])) ** 16.)   # The super-gaussian event

    sine = params["Steady Production"] + \
        0.18 * params["Steady Production"] * \
        sin(2 * pi / 11 * t + params["Phase (yr)"]["mean"]) # Sinusoidal component of production 
    
    return sine + gauss

In [ ]:
def get_residual_distribution(model: ticktack.fitting.SingleFitter, params: list):
    """
    Parameters:
        production: function - The production function, typically determined using `get_production_function`.
        data: str - The file name of the data. Should be the same as the file name that was provided to `get_production_function`.
    Returns:
        An `mcmc` sample of the posterior distribution of the residuals, which has been fitted with a parameteric (to start) distribution that can be used to simulate noise.
    """
    # Not needed right away. 
    # DFM's is the maximum level IR
    # The simplest way:
        # For each year take a 50 (flexible) year chunk of real data from Intcal20.
        # We do not want to use Intcal20 because the data is interpolated to be on the same grid although they are not. 
        # We have that the data. #! Get of Uratkash
        #? Is there a spike in the midyear?
        # We determine this be comparing to the ideal simulated data (Template)
        # Calculate the difference in chi sqaured between the data and a perfectly straight line and the simulated event. 
            #* Long term increase can be fitted with linear regression.  
            # Long term might use optimisation, gp or ea

    # Evolutionary algorithms are a method of optimasation.
        # Imagine there is a bunch of different combinations of parameters that map to a genome. calculate the objective function for each genome. which is the fitness of the model. The ones with the worst performance are killed breading the best performing models.
        # Efficient for difficult optimisation problems.
        # Hybrid fitness through iteration. 
        # Alsiong wong
    
    # Gaussian processes are also useful in optimisation
        # A way of representing on-parametric function as a kernel is a distribution over functions. (Hilber space?)
        # The covariance matrix of the multi-dimensional sampling space is parametrised (as the kernel). 
        # The covariance is itself a function of the independent variable. 
        # About the statistics of functions

    #! The Mackie book has chapters on both
    # There is also differential evolution. 

    # Bayesian information criterion and the Akaike tell us the worth of additional parameters. Callibrate for each year for computational feesible run times. Resample the noise and see how often you get different chi squared differences / BICs and Akaikes. This is a histogram of the test statistic and get the 95% confidence level and how often would there be a false positive. Both false positives and false negitives are of interest. (The independent variable is the event size.)

    # ?Bootstrapping

    # Using this table of the test statistic. What test stat value is the best?
    # Repeat this for every year and therefore know what event size to look for in each year. 
    # This will be a big grid. This is a table of sensitivities. For each event given its parameters calculate the probablity of finding that event. If one has a 0.5 chance then there were two of them.

    #? Per Bak has a book "How nature works: The science of self organised criticality."
    
    # The dist of miyake events will have some distribution that might be one over f
    # This will mean that the 6 main events tell you something about the distribution of small events. 
    # Ultimately this is where we want to go.

    residuals = model.d14c_data - model.dc14(params)

    gaussian_error_parameters = {   # Dictionary containing the parameters of a parametric gaussian
        "mean": mean(residuals),    # The mean of the residuals assumed to have gaussian error
        "variance": var(residuals)  # Variance of the residuals assumed to have gaussian error
    }

    return gaussian_error_parameters
    

So the error in the data measurements will have a distribution that I can use to generate imaginary error in the data. I'm not sure how this will help but it could change the way things shape out so I will implement this and keep track of it. 

In [ ]:
def simulate_error(model: ticktack.fitting.SingleFitter, error: dict, theoretical_dc14: array):
    """
    Simulates a Miyake event based on the things that have already transpired. 
    """   
    size = len(model.d14c_data) # Size of the model 
    random_error = random.randn(size) * error["variance"] + error["mean"] # Generating the noise 

    data = {    # A Dictionary that I will convert to a DataFrame and return 
        "Year": model.time_data,        # The time series data 
        "DC14": theoretical_dc14 + random_error,           # Simulated C14 data
    }

    return DataFrame(data)

In [ ]:
def recover_event(event: DataFrame, EDC14: array):
    """
    Used to determine how likely an event is to have occured. 
    Parameters:
        event: `DataFrame` -> A simulated event
        EDC14: `array` -> The theoretical values 
    Returns:
        The chi-squared of the simulated event given the EDC14 and the error in the chi-squared
    """
    # The first line should be a linear best fit
    # Then calculate the chi squared and the test statistic is the difference between that and the template.

    chi_squared = (array([*event["DC14"]]) - EDC14) ** 2 / array([*event[sig_DC14]]) ** 2  # Calculates generalised chi squared
    # chi_squared_error = 2 * array([*event["Sig_DC14"]])   # Error propagation of chi-sqaured 
    return float(sum(chi_squared))

In [ ]:
test_models = {model: models[model] for model in ["Guttler14", "Miyake17"]} # Extracting two test models
test_data = data_sets[:1]   # Test data sets to run the test models on

In [ ]:
simulations = {         # Holds the data farmed from the simulation 
    "Model": [],        # The model that was used in the simulation
    "Data": [],         # Holds the name of the dataset
    "Height": [],       # The peak height of the production function 
    "Duration": [],     # The duration of the productioin function 
    "Chi Squared": []   # The chi-squared of the model 
}

for model in test_models:   # Looping over the models 
    for datum in test_data: # Looping over the data
        datum_str = datum.split("/")[-1]    # Just storing the csv name not the whole adress path 
        
        print(  
            f"Current Model = {model} \n" +
            f"Current Datum = {datum_str}"
        )

        model_obj = get_model(model, datum) # Loading the model into RAM
        model_obj.production = production   # Assigning the production function 

        production_params = get_production_function(model_obj)   # Getting the production function 
        data_error = get_residual_distribution(model_obj, [production_params])  # Getting error 

        for height in arange(0.1, 8.0, 0.1):   # Looping over a range of areas 
            for duration in arange(0.1, 8.0, 0.1):   # Looping over a range of durations
                event_params = production_params.copy()             # Copying the parameters 
                event_params["Area"]["mean"] = height * duration    # Updating the area parameter
                event_params["Duration (yr)"]["mean"] = duration    # Changing the duration 

                theoretical_dc14 = model_obj.dc14([event_params])      # Running the model 
                event = simulate_error(model_obj, data_error, theoretical_dc14)  # Running simulation 
                chi_squared = recover_event(event, theoretical_dc14) # getting shi 

                simulations["Data"].append(datum_str)           # Storing the data
                simulations["Model"].append(model)              # Storing the model 
                simulations["Height"].append(height)            # Storing the simulation area
                simulations["Duration"].append(duration)        # Storing the simulations duration 
                simulations["Chi Squared"].append(chi_squared)  # Storing the chi-squared

        print("\n") # New line for nicer terminal display 

So it is starting to come together. I need to work out what is causing the huge variance in the chi-squared though. I might do this on some smaller meshes. 
I also might want to plot all the data sets so that I can see what I am dealing with. I'll make this the first goal for after lunch.

In [ ]:
simulations = DataFrame(simulations)
(ggplot(simulations, aes(x="Height", y="Duration", fill="Chi Squared"))
    + geom_tile()
    + scale_color_cmap(cmap_name="Spectral")
    + facet_wrap("~Model + Data"))

In [ ]:
simulations

Kernel density information will enable me to get the maximum likelhood as well as the mean and median.